## <center>CSE 546: Reinforcement Learning</center>
### <center>Prof. Alina Vereshchaka</center>
<!-- ### <center>Fall 2022</center> -->

Welcome to the Assignment 2, Part 1: Introduction to Deep Reinforcement Learning and Neural Networks! The goal of this assignment is to make you comfortable with the application of different Neural Network structures depending on how the Reinforcement Learning environment is set up.

## Disabling Warnings

In [11]:
import warnings
warnings.filterwarnings("ignore")

## Installing Required Libraries

In [12]:
%pip install torch opencv-python gymnasium matplotlib numpy


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Imports

In [13]:
from environment import WumpusWorldEnvironment
import torch
import torch.nn as nn

# Environment

We will be working with an implementation of the Wumpus World environment. The environment comes from the book "Artificial Intelligence: A Modern Approach" by Stuart J. Russell and Peter Norvig. 

### ENVIRONMENT DETAILS:

The environment is a 6 x 6 grid world containing a total of 36 grid blocks. 

#### ENVIRONMENT OBJECTS:
The environment consists of the following objects:

1. **Agent** - The agent starts in the grid block at the bottom left corner whose co-ordinates are [0, 0]. The goal of our agent is to collect the Gold while avoiding the Wumpus and the pits. 

2. **Wumpus** - The monster which would eat the agent if they are in the same grid block.

3. **Pit** - The agent must avoid falling into the pits. 

4. **Gold** - The agent must collect the Gold.

5. **Breeze** - Breeze surrounds the Pits and warn the agent of a Pit in an adjacent grid block.

6. **Stench** - Stench surrounds the Wumpus and warns the agent of the Wumpus in an adjacent grid block.

#### ENVIRONMENT OBSERVATIONS:

Our implementation of the environment provides you with four different types of observations:

1. **Integer** - Integer in the range [0 - 35]. This represents the grid block the agent is in. E.g., if the agent is in the bottom left grid block (starting position) the observation would be 0, if the agent is in the grid block containing the Gold the observation would be 34, if the agent is in the top right grid block the observation would be 35.

2. **Vector** - 

    **2.1.** A vector of length 2 representing the agent co-ordinates. The first entry represents the x co-ordinate and the second entry represets the y co-ordinate. E.g., if the agent is in the bottom left grid block (starting position) the observation would be [0, 0], if the agent is in the grid block containing the Gold the observation would be [4, 5], if the agent is in the top right grid block the observation would be [5, 5].
    
    **2.2.** A vector of length 36 representing the one-hot encoding of the integer observation (refer type 1 above). E.g., if the agent is in the bottom left grid block (starting position) the observation would be [1, 0, ..., 0, 0], if the agent is in the grid block containing the Gold the observation would be [0, 0, ..., 1, 0], if the agent is in the top right grid block the observation would be [0, 0, ..., 0, 1].


3. **Image** - Image render of the environment returned as an NumPy array. The image size is 84 * 84 (same size used in the DQN paper). E.g., if the agent is in the bottom right grid block the observation is:

    Observation: (84 * 84)

     [[255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     ...

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]]

    Observation type: <class 'numpy.ndarray'>

    Observation Shape: (84, 84)

    Visually, it looks like:
    <img src="./images/environment_render.png" width="500" height="500">
    

4. **Float** - Float in the range [0 - $\infty$] representing the time elapsed in seconds. 

#### ENVIRONMENT ACTIONS:

Our implementation of the environment provides you with three different types of actions:

1. **Discrete** - Integer in the range [0 - 3] representing the four actions possible in the environment as follows: 0 - Right 1 - Left 2 - Up 3 - Down.

2. **Multi-Discrete** - Array of length four where each element takes binary values 0 or 1. Array elements represent if we take a particular action. Array element with index 0 corresponds to the right action, index 1 corresponds to the left action, index 2 corresponds to the up action, and index 3 corresponds to the down action. E.g., 
   action = [1, 0, 0, 0] would result in the agent moving right.
   action = [1, 0, 1, 0] would result in the agent moving right and up.
   action = [0, 1, 0, 1] would result in the agent moving left and down.

3. **Continuous** - Float in the range [-1, 1] determining whether the agent will go left, right, up, or down as follows:

    if -1 <= action <= -0.5:
        Go Right.
    elif -0.5 < action <= 0:
        Go Left.
    elif 0 < action <= 0.5:
        Go Up.
    elif 0.5 < action <= 1:
        Go Down.
        
### YOUR TASK IS TO USE A NEURAL NETWORK TO WORK WITH ALL FOUR TYPES OF OBSERVATIONS AND ALL THREE TYPES OF  ACTIONS.
### Note: You don't have to train your agent/neural network. You just have to build the neural network structure that takes the observation as input and produces the desired output with the initial weights.

#### You can use libraries such as PyTorch/TensorFlow/Keras to build your neural networks.

#### <span style="color:red">You cannot use RL libraries that already provide the neural network to you such as Stable-baselines3, Keras-RL, TF agents, Ray RLLib etc.</span>

<img src="./images/wumpus_world_environment.jpg" width="600" height="600">

# START COMPLETING YOUR ASSIGNMENT HERE

## Observation Type - Integer, Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is an integer (refer to environment observations type 1). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_1_64_4.png">

In [77]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class QNN1(nn.Module):
    def __init__(self):
        super(QNN1, self).__init__()
        self.nn1 = nn.Linear(36, 64)
        self.nn2 = nn.Linear(64, 4)
    
    def forward(self, x):
        x = torch.relu(self.nn1(x))
        x = self.nn2(x)
        return x

observation_one_hot = torch.zeros(36)
observation_one_hot[observation] = 1.0
observation_tensor = observation_one_hot.unsqueeze(0)

model = QNN1()
q_values = model(observation_tensor)

predicted_action = torch.argmax(q_values, dim=1).item()

print("Observation:", observation)
print("Q-values :", q_values.detach().numpy())
print("Predicted action:", predicted_action)
# END_YOUR_CODE

Observation: 0
Q-values : [[ 0.00356039 -0.10186707  0.06063999 -0.02941073]]
Predicted action: 2


In [78]:
print("Observation space:", environment.observation_space)

Observation space: Discrete(36)


## Observation Type - Vector (2.1), Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 2 (refer to environment observations type 2.1). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_2_64_4.png">

In [82]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='vector', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class QNN2(nn.Module):
    def __init__(self):
        super(QNN2,self).__init__()
        self.nn1 = nn.Linear(2,64)
        self.nn2 = nn.Linear(64,4)
    
    def forward(self, x):
        x = torch.relu(self.nn1(x))
        x = self.nn2(x)
        return x
    
observation_tensor = torch.tensor([observation], dtype=torch.float32)

model = QNN2()
q_values = model(observation_tensor)

predicted_action = torch.argmax(q_values, dim=1).item()
print("Observation:", observation)
print("Q-values:", q_values.detach().numpy())
print("Predicted action:", predicted_action)

# END_YOUR_CODE

Observation: [0 0]
Q-values: [[ 0.01801422 -0.04489758  0.04355751  0.09099082]]
Predicted action: 3


In [83]:
print("Observation space:", environment.observation_space)

Observation space: Discrete(36)


## Observation Type - Vector (2.2), Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector.

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_4.png">

In [90]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class QNN3(nn.Module):
    def __init__(self):
        super(QNN3,self).__init__()
        self.nn1 = nn.Linear(36,64)
        self.nn2 = nn.Linear(64,4)
    
    def forward(self, x):
        x = torch.relu(self.nn1(x))
        x = self.nn2(x)
        return x
    
observation_one_hot = torch.zeros(36)
observation_one_hot[observation] = 1.0
observation_tensor = observation_one_hot.unsqueeze(0)

model = QNN3()
q_values = model(observation_tensor)

predicted_action = torch.argmax(q_values, dim=1).item()

print("Observation:", observation_tensor)
print("Q-values:", q_values.detach().numpy())
print("Predicted action:", predicted_action)

# END_YOUR_CODE

Observation: tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
Q-values: [[ 0.02585754  0.1335594  -0.12394436  0.1293357 ]]
Predicted action: 1


## Observation Type - Image, Action Type - Discrete

The part of the assignment requires you to create a convolutional neural network with one convolutional layer having 128 filters of size 3 x 3, one hidden layer having 64 neurons, and the output layer having 4 neurons. The input to the neural network is an image of size 84 * 84 (refer to environment observations type 3). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/convolutional_neural_network_84x84_128_64_4.png">

In [92]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='image', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 84 * 84, 64)
        self.fc2 = nn.Linear(64, 4)
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

observation_tensor = torch.tensor(observation, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

model = CNN()

q_values = model(observation_tensor)
predicted_action = torch.argmax(q_values, dim=1).item()

print("Observation shape:", observation.shape)
print("Observation", observation_tensor)
print("Q-values:", q_values.detach().numpy())
print("Predicted action:", predicted_action)

# END_YOUR_CODE

Observation shape: (84, 84)
Observation tensor([[[[255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          ...,
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.]]]])
Q-values: [[-14.191583     0.46344006  38.310146   -17.309221  ]]
Predicted action: 2


## Observation Type - Float, Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a float (refer to environment observations type 4). The output of the neural network is an array representing the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_1_64_4.png">

In [97]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""


environment = WumpusWorldEnvironment(observation_type='float', action_type='discrete')
observation, info = environment.reset()


# BEGIN_YOUR_CODE
class QNNFloat(nn.Module):
    def __init__(self):
        super(QNNFloat, self).__init__()
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 4)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

observation_tensor = torch.tensor([observation], dtype=torch.float32).unsqueeze(0)

model = QNNFloat()

q_values = model(observation_tensor)

print("Observation:", observation)
print("Q-values:", q_values.detach().numpy())

# END_YOUR_CODE

Observation: 6.389617919921875e-05
Q-values: [[ 0.1057041   0.19957788 -0.04125413  0.25671482]]


## Observation Type - Vector (2.2), Action Type - Multi-Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an array representing the probability of choosing the actions. (If the value of the array element is >=0.5 you will perform the action.) (refer to environment actions type 2).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector.

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_4_sigmoid.png">

In [117]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted action probabilities for the four actions. Print the observation and the action probabilities."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='multi_discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class ActionProbabilityNN(nn.Module):
    def __init__(self):
        super(ActionProbabilityNN, self).__init__()
        self.fc1 = nn.Linear(36, 64)
        self.fc2 = nn.Linear(64, 4)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

observation_one_hot = torch.zeros(36)
observation_one_hot[observation] = 1.0
observation_tensor = observation_one_hot.unsqueeze(0)

model = ActionProbabilityNN()

action_probabilities = model(observation_tensor)

predicted_action = torch.argmax(action_probabilities, dim=1).item()

print("Observation:", observation_tensor)
print("Action probabilities:", action_probabilities.detach().numpy())
print("Predicted action:", predicted_action)

# END_YOUR_CODE

Observation: tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
Action probabilities: [[0.4842266  0.4889721  0.4986303  0.51958686]]
Predicted action: 3


## Observation Type - Vector (2.2), Action Type - Continuous

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 1 neuron. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an float in the range [-1, 1] determining the action which will be taken. (refer to environment actions type 3).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector and use the TanH activation function to get the output in the range [-1, 1].

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_1.png">

In [113]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted action. Print the observation and the predicted action."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='continuous')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class ContinuousActionNN(nn.Module):
    def __init__(self):
        super(ContinuousActionNN, self).__init__()
        self.fc1 = nn.Linear(36, 64)
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        return x

observation_one_hot = torch.zeros(36)
observation_one_hot[observation] = 1.0
observation_tensor = observation_one_hot.unsqueeze(0)

model = ContinuousActionNN()

predicted_action = model(observation_tensor)

if -1 <= predicted_action <= -0.5:
    predicted_action_value = 0
elif -0.5 < predicted_action <= 0:
    predicted_action_value = 1
elif 0 < predicted_action <= 0.5:
    predicted_action_value = 2
elif 0.5 < predicted_action <= 1:
    predicted_action_value = 3


print("Observation:", observation_tensor)
print("Predicted action value:", predicted_action.detach().numpy())
print("Predicted action:", predicted_action_value)

# END_YOUR_CODE

Observation: tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
Predicted action value: [[-0.06756126]]
Predicted action: 1
